In [6]:
from PIL import Image
import numpy as np
import os

In [7]:
# Configuration
# macOS 下路径写法可能需要调整

# 已生成的 Playdate 字体图片路径
FONT_IMAGE_TABLE_FILEPATH = "rabbit-table-105-105.png"
# 输出文件夹路径
OUTPUT_DIR = 'output'

# 旋转方向：
# rotate_right_90   顺时针旋转 90°
# rotate_left_90    逆时针旋转 90°
# rotate_180        旋转 180°
ROTATE_DIRECTION = "rotate_right_90"
# 跳过前多少个字符不进行旋转。如果使用 make_font.ipynb 生成的字体，设置为 101 可以跳过半角拉丁字母与符号
SKIP_ROTATION = 0

In [8]:
FONT_GRID_WIDTH_HEIGHT = os.path.basename(FONT_IMAGE_TABLE_FILEPATH).replace(".png", "").split("-table-")[1:][0].split("-")

In [9]:
def rotate_img_cells(img_path):
    img = Image.open(img_path)
    width, height = img.size

    cell_width = int(FONT_GRID_WIDTH_HEIGHT[0])
    cell_height = int(FONT_GRID_WIDTH_HEIGHT[1])
    
    # 将图片转为numpy数组
    img_array = np.array(img)

    skip_count = 0
    
    # 从左到右，从上到下顺时针旋转90度每个格子内的像素
    for i in range(0, height, cell_height):
        for j in range(0, width, cell_width):
            if skip_count < SKIP_ROTATION:
                skip_count += 1
                continue

            cell = img_array[i:i + cell_height, j:j + cell_width]
            if ROTATE_DIRECTION == "rotate_right_90":
                cell_rotated = np.flip(np.transpose(cell, (1, 0, 2)), 1)
            elif ROTATE_DIRECTION == "rotate_left_90":
                cell_rotated = np.flip(np.transpose(cell, (1, 0, 2)), 0)
            elif ROTATE_DIRECTION == "rotate_180":
                cell_rotated = np.flip(np.flip(cell, 0), 1)
            img_array[i:i + cell_height, j:j + cell_width] = cell_rotated
            
    # 保存结果
    rotated_img = Image.fromarray(img_array)
    rotated_img.save(os.path.join(OUTPUT_DIR, os.path.basename(img_path)))

In [10]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

rotate_img_cells(FONT_IMAGE_TABLE_FILEPATH)